In [ ]:
#Reference Link https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

In [1]:
from selenium import webdriver

# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
#DRIVER_PATH = r"C:\Users\conor\Desktop\Scraping\chromedriver_win32\chromedriver.exe"
wd = webdriver.Chrome(executable_path= r"C:\Users\conor\Desktop\Scraping\chromedriver_win32\chromedriver.exe")

In [14]:
wd.get('https://google.com')

In [15]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dogs')

In [2]:
wd.quit()

In [12]:
from selenium import webdriver
import os
import time
import io
import requests
from PIL import Image
import hashlib

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [13]:
import PIL
import os
import time
import io
import requests
from PIL import Image
import hashlib

def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [14]:
import os
import time
import io
import requests
from PIL import Image
import hashlib

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [19]:
import os
import time
import io
import requests
from PIL import Image
import hashlib

search_term1 = "sedan car shape"
search_term2 = "hatchback car shape"
TARGET_PATH1 = r"C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Sedan_Images"
TARGET_PATH2 = r"C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images"

search_and_download(search_term = search_term2, driver_path = r"C:\Users\conor\Desktop\Scraping\chromedriver_win32\chromedriver.exe", target_path = TARGET_PATH2, number_images = 120)

Found: 200 search results. Extracting links from 0:200
Found: 120 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRnTJAbQ0vUgP5qfpG2pI5wSHIgOXsbcwgnHK22iUGeN6B75Z9u&usqp=CAU - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\80e30050bb.jpg
SUCCESS - saved https://en.pimg.jp/059/888/306/1/59888306.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\d8c17e0763.jpg
SUCCESS - saved https://c0.thejournal.ie/media/2017/01/nissan_pulsar_4-2-390x285.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\3d1db0b7c7.jpg
SUCCESS - saved https://static.techspot.com/images2/news/bigimage/2019/09/2019-09-11-image.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\dc5c6a1cf9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=t

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQkczYKeQMEqvmxzdIu57QZ6tPlxQ5OYkd3AB-x6LLUR2TtdsDO&usqp=CAU - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\41368c0f07.jpg
SUCCESS - saved https://ae01.alicdn.com/kf/HTB1PbPdp21TBuNjy0Fjq6yjyXXaR/For-Lowered-car-shape-sticker-based-on-Subaru-Impreza-WRX-STI-Hatchback-GH-2007-Car-Styling.jpg_q50.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\4c0b284430.jpg
SUCCESS - saved https://cdn.powered-by-nitrosell.com/product_images/12/2821/large-10083827%20OASIS%20FOAM%20FRAMES%20Modern%20Motor%20Car.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\d23ef2d355.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT9HQD7Z77R2ub3bl5E59szIBmxW1dCAR6M56FWExGbRbTQdcr3&usqp=CAU - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignme

ERROR - Could not save https://9clouds.com/wp-content/uploads/2017/09/More-Cars-Connected-to-Internet-than-Smartphones-in-2016.png - cannot identify image file <_io.BytesIO object at 0x000001AECA215A98>
SUCCESS - saved https://cdn.motor1.com/images/mgl/QXMj1/s1/2015-591707peugeot-fractal-concept1.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\2c39c9258f.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/2018_Toyota_Aygo_x-play_1.0.jpg/1200px-2018_Toyota_Aygo_x-play_1.0.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\88ae5a3f79.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRR7VZbFNX0WbKnlfc1zth3a4sROAtfHS-TG3PjQD339gw0XsN2&usqp=CAU - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\ffcd8d076b.jpg
SUCCESS - saved https://cdn1.carbuyer.co.uk/sites/carbuyer_d7

SUCCESS - saved https://images-na.ssl-images-amazon.com/images/I/51YmSuNurJL._SL1036_.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\b54ec8ac1b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT0wZvG30Noc6IojotoGXBcu_uQ203IgNEHFIHIE1hwGfZZlaaw&usqp=CAU - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\d45387bbc4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT_aOdhOhCz0BmCsyKj7z9TI_bCV2w9V5tX0bqyQzTG3Md8wP5Y&usqp=CAU - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\59331e07c6.jpg
SUCCESS - saved https://c8.alamy.com/comp/E859CH/three-dimensional-shape-green-sedan-studio-shot-E859CH.jpg - as C:\Users\conor\Documents\ResearchTopicsInAI\Assignment2\Images\Hatchback_Images\hatchback_car_shape\b54d2c563c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?